Plot attention maps for transformers models

In [64]:
import json
from pathlib import Path

import torch
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
repo_root = Path(r'C:\Users\apeganov\DeepLearningExamplesLocal')
fixed_emb_attn_path_rel_2_root = Path(
    'PyTorch/LanguageModeling/Transformer-XL/ngc_results/fixed_pos_emb_mem_fp32/attn/1455276')
fixed_emb_attn_path_abs = repo_root / fixed_emb_attn_path_rel_2_root

def get_attn(path):
    attn_path = path / Path('attn.npy')
    keys_path = path / Path('keys.npy')
    queries_path = path / Path('queries.npy')
    with attn_path.open('rb') as af, keys_path.open('rb') as kf, queries_path.open('rb') as qf:
        return {'attn': np.load(af), 'keys': np.load(kf), 'queries': np.load(qf)}

step1 = get_attn(fixed_emb_attn_path_abs / Path('step1'))

print('attn.shape:', step1['attn'].shape)
print('keys.shape:', step1['keys'].shape)
print('queries.shape:', step1['queries'].shape)

In [ ]:
def plot_attention_weights(attentions, tokens, layer, filename='att.png', save=False):
    # attentions n_layers x n_heads x len x len
    fig = plt.figure(figsize=(30, 90))
    
    attention = attentions[layer]
  
    for head in range(attention.shape[0]):
        ax = fig.add_subplot(6, 2, head+1)

        # plot the attention weights
        ax.matshow(attention[head][:len(tokens),:len(tokens)], cmap='Reds')

        fontdict = {'fontsize': 20}

        ax.set_xticks(range(len(tokens)))
        ax.set_yticks(range(len(tokens)))

        ax.set_xticklabels(tokens, 
            fontdict=fontdict, rotation=90)

        ax.set_yticklabels(tokens,
            fontdict)

        ax.set_xlabel('Head {}'.format(head+1), fontdict)

    plt.tight_layout()
    if save:
        plt.savefig(filename)
    plt.show()

In [3]:
def plot_cls_attention_weights(attentions, tokens, filename='att_cls.png', save=False):
    # attentions n_layers x n_heads x len x len
    fig = plt.figure(figsize=(30, 40))
    ax = fig.add_subplot(1, 1, 1)
    cls_attentions = attentions[:,:,0,:len(tokens)].view(-1, len(tokens))
    ax.matshow(cls_attentions, cmap='Reds')
    fontdict = {'fontsize': 20}

    ax.set_xticks(range(len(tokens)))
    ax.set_yticks(range(cls_attentions.shape[0]))

    ax.set_xticklabels(tokens, fontdict=fontdict, rotation=90)

    ax.set_yticklabels([f'layer {i} head {j+1}' for i in range(attentions.shape[1]) for j in range(attentions.shape[0])], fontdict)

    plt.tight_layout()
    plt.title('[CLS] attention map')
    if save:
        plt.savefig(filename)
    plt.show()

In [68]:
def compute_mean_att_scores(attentions, tokens, mean=True, att_scores=None):
    if att_scores is None:
        att_scores = pd.DataFrame([[[] for i in range(3)] for j in range(3)])
        att_scores.columns = ['special', 'mem', 'regular']
        att_scores.index = ['special', 'mem', 'regular']

    tokens_map = {
        '[CLS]': 'special',
        '[SEP]': 'special',
        '[MEM]': 'mem',
    }

    for i in range(attentions.shape[2]):
        i_token_type = tokens_map.get(tokens[i], 'regular')
        for j in range(attentions.shape[3]):
            j_token_type = tokens_map.get(tokens[j], 'regular')
            att_scores[i_token_type][j_token_type] += attentions[:,:,i,j].flatten().tolist()
    if mean:
        att_scores = att_scores.applymap(np.mean)
    return att_scores

In [5]:
#config = json.load(open('/home/yurakuratov/.deeppavlov/models/classifiers/glue_mrpc_torch_cased_bert_bert-base-uncased/config.json', 'r'))
config = json.load(open('/home/yurakuratov/data/DeepPavlov/deeppavlov/configs/classifiers/glue/glue_mrpc_cased_bert_torch_mem.json', 'r'))
#config = json.load(open('/home/yurakuratov/data/DeepPavlov/deeppavlov/configs/classifiers/glue/glue_mnli_cased_bert_torch_mem.json', 'r'))
#config = json.load(open('/home/yurakuratov/.deeppavlov/models/classifiers/glue_mrpc_torch_cased_bert_mem_15_pool_mem/config.json', 'r'))
config = json.load(open('/home/yurakuratov/.deeppavlov/models/classifiers/glue_qnli_torch_cased_bert_mem_5_pool_mem/config.json', 'r'))
config = json.load(open('/home/yurakuratov/.deeppavlov/models/classifiers/glue_qnli_torch_cased_bert_mem_10_pool_mem/config.json', 'r'))

In [6]:
def change_component(config, component, field, value):
    value_set = False
    for i in range(len(config['chainer']['pipe'])):
        if config['chainer']['pipe'][i]['class_name'] == component:
            config['chainer']['pipe'][i][field] = value
            value_set = True
            break
    return config, value_set

In [7]:
config, _ = change_component(config, 'torch_bert_classifier', 'output_attentions', True)
config, _ = change_component(config, 'torch_bert_classifier', 'out', ['y_pred_probas', 'attentions'])
##################################
#mem_size = 10
#config, _ = change_component(config, 'torch_mem_tokens_preprocessor', 'mem_size', mem_size)
#config['metadata']['variables']['MODEL_PATH'] += f'_{mem_size}'

In [8]:
model = build_model(config)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/yurakuratov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yurakuratov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/yurakuratov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/yurakuratov/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-09-10 19:39:25.571 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/yurakuratov/data/deeppavlov_download/models/classifiers/glue_qnli_torch_cased_bert_mem_10_pool_mem/classes.dict]
2020-09-10 19:39:29.770 INFO in 'deeppavlov.models.torch_bert.torch_bert_classifier'['torch_bert_classifier'] at line 222: Embeddings matrix size was c

In [9]:
from deeppavlov.models.preprocessors.torch_bert_preprocessor import TorchBertPreprocessor
tokenizer = None
for el in model.pipe:
    if isinstance(el[-1], TorchBertPreprocessor):
        tokenizer = el[-1]

In [69]:
from nlp import load_dataset
#dataset = load_dataset('glue', 'mnli', split=['validation_matched'])
dataset = load_dataset('glue', 'qnli', split=['validation'])

n = 100
att_scores = None
for i, example in tqdm(enumerate(dataset[0])):
    if i >= n:
        break
    h = example['question']#example['sentence1']#example['hypothesis']
    p = example['sentence']#example['sentence2']#example['premise']
    bert_features, attentions = model.compute([[h, p]], targets=['bert_mem_features', 'attentions'])
    bert_features = bert_features[0]
    attentions = attentions[0]
    attentions = torch.stack(attentions, dim=1).squeeze(dim=0).cpu()
    tokens = [tokenizer.tokenizer.ids_to_tokens.get(t_id, '[MEM]') for t_id in bert_features.input_ids[0].tolist() if t_id]
    # cut all paddings
    attentions = attentions[:,:,:len(tokens),:len(tokens)]
    att_scores = compute_mean_att_scores(attentions, tokens, mean=False, att_scores=att_scores)

print('mean attention read/write scores:')
att_scores.applymap(np.mean)

100it [00:38,  2.57it/s]


mean attention read/write scores:


,special,mem,regular
special,0.221538,0.114856,0.117944
mem,0.002992,0.017776,0.002454
regular,0.005994,0.009373,0.010150


### plot attention maps for single example

In [ ]:
h, p

In [ ]:
#sentence1 = "Wal-Mart said it would check all of its million-plus domestic workers to ensure they were legally employed ."
#sentence2 = "It has also said it would review all of its domestic employees more than 1 million to ensure they have legal status ."
#sentence1="I wont even bother listening."
#sentence2="i'll listen  and agree with what i think sounds right"
sentence1 = h
sentence2 = p
bert_features, attentions = model.compute([[sentence1, sentence2]], targets=['bert_mem_features', 'attentions'])
bert_features = bert_features[0]
attentions = attentions[0]

In [ ]:
attentions = torch.stack(attentions, dim=1).squeeze(dim=0).cpu()
attentions.shape

In [ ]:
tokens = [tokenizer.tokenizer.ids_to_tokens.get(t_id, '[MEM]') for t_id in bert_features.input_ids[0].tolist() if t_id]
# cut all paddings
attentions = attentions[:,:,:len(tokens),:len(tokens)]

In [ ]:
plot_cls_attention_weights(attentions, tokens=tokens)

In [ ]:
for layer in range(attentions.shape[0]):
    print(f'layer: {layer}')
    plot_attention_weights(attentions, tokens=tokens, layer=layer)